# FFT (Fast Fourier Transform)

The following pieces of code will define the FFT, which uses many different algorithms to compute the DFT of a list (or a matrix, which shall be discussed later) of data, defined as:

$X_k = \sum^{N-1}_{n=0}x_n ⋅ e^{-iτkn/N}, \;N ≜ $ size of $x$

(It may also be written as: 
$X_k = \sum^{N-1}_{n=0}x_n ⋅ ω^{kn}, \; N$ ≜ size of $x$, $ω$ ≜ $e^{-iτ/N}$
)

depending on the size of the input list, as follows:

If $N$ is a power of 2, use a radix 2 FFT, else if $N$ is a prime number, use a rader FFT, else use a bluestein FFT.

These algorithms have a big $O$ notation of $O$($ N \cdot log(N) $), whereas manually computing the DFT has a big $O$ notation of $O$($N^2$). (I.e. as $N$ is scaled by some $k$, the time complexity of FFT algortithms increase by $k \cdot log(k)$, whereas the time complexity of a DFT increases by $k^2$).

This makes using these algorithms much more obvious than manual computation of the DFT, hence why they are so useful to us.


The following cell defines some starting code

In [1]:
using Primes

τ = 2π

# Displays a vector after formatting it (makes logs easier to read)
function format_display(x::Union{Vector{T}, Matrix{T}})::Nothing where T <: Real
    display(map(elem -> round(elem, digits=2), x))
end

# Displays a matrix after formatting it (makes logs easier to read)
function format_display(x::Union{Vector{T}, Matrix{T}})::Nothing where T <: Number
    display(map(elem -> round(real(elem), digits=2) + round(imag(x), digits=2)))
end

format_display (generic function with 2 methods)

## Radix 2 FFT 

The following code cell defines a radix 2 FFT

When N is a power of 2, one can rearrange the list by reversing the indices digits (in base 2). One can then recursively compute the FFT butterfly algorithm to reconstruct the desired FFT.

This can be simplified to: 

$FFT(x) = FFT(x_e) + (FFT(x_o) \cdot U^{(N/2)-1}_{n=0}ω^n) \; ⌢ \; FFT(x_e) - (FFT(x_o) \cdot U^{(N/2)-1}_{n=0}ω^n)$

where $U_{r=a}^bf(r) = [f(a), f(a+1), ..., f(b-1), f(b)]$ and $⌢$ is concatenation.

It is thus clear to see that a radix 2 FFT is recursive, and this allows for a much faster computation than manually computing a DFT.

To ensure a StackOverflow error doesn't occur, we need a base case where the recursion should stop, which would be when $N = 1$, because then there will only be one element, and thus $n = 0$, and so the complex exponential component is just 1.

For reasons to be made clear later (see radix 2 IFFT), ω shall be parameterised (to allow different values to be supplied). (By default, it will have a value of nothing, then will be initialised in the function).

In [8]:
function radix2FFT(x::Vector{T}, ω::Union{Complex{A}, Nothing} = nothing)::Vector{ComplexF64} where {T <: Number, A <: Number}
    N = length(x)
    
    if N == 1
        return x # One-point time domain spectrum is the same in frequency domain
    end

    if !ispow2(N)
        throw(ArgumentError("Input list must have a length of a power of 2"))
    end

    # ω ≜ exp(-im*τ/N) (if not already supplied, see IFFT for when different ω used) 
    if ω === nothing
        ω = exp(-im*τ/N)
    end    

    Xₑ = radix2FFT(x[1:2:end], ω^2) # Get the even indices of x (0-indexed) and recursively FFT (Julia is 1-indexed)
    Xₒ = radix2FFT(x[2:2:end], ω^2) # Get the odd indices of x (0-indexed) and recursively FFT (Julia is 1-indexed)

    res = zeros(ComplexF64, N)
    
    for i ∈ 1:N÷2
        res[i] = Xₑ[i] + Xₒ[i] * ω^(i-1) 
        res[i+N÷2] = Xₑ[i] - Xₒ[i] * ω^(i-1)
    end

    return res
end

radix2FFT (generic function with 2 methods)

## Radix 2 IFFT
The following code cell defines a radix 2 IFFT

The Inverse Discrete Fourier Transform is defined to be:

$x_n = \sum_{k=0}^{N-1}X_k ⋅ ω^{-kn/N}$

Notice how this definition is very similar to that of the FFT, just that the exponent is the negative of the exponent in the FFT. This can be leveraged as follows:

$IFFT(X) = FFT(X,\; ω = e^{iτ/N})$

hence why we parameterised ω when writing the code to do the FFT.

We can also use this fact when making a rader_IFFT and a bluestein_IFFT (later on).

In [10]:
function radix2IFFT(x::Vector{T}, ω::Union{Complex{U}, Nothing} = nothing)::Vector{ComplexF64} where {T <: Number, U <: Number}
    N = length(x)

    if N == 1
        return x # One-point time domain spectrum is the same in frequency domain
    end

    if !ispow2(N)
        throw(ArgumentError("Input list must have a length of a power of 2"))
    end

    # Initialise root of unity to be used
    if ω === nothing
        ω = exp(τ*im/N)
    end

    return radix2FFT(x, 1/ω) ./ N # Supply 1/ω because exp(-iτ/N) = 1/exp(iτ/N)
end

radix2IFFT (generic function with 2 methods)

## Rader FFT

The Rader FFT can be used on prime length inputs because if $N$ (the length of the inputs is prime), the following is true:

$(\frac{\mathbb{Z}}{N\mathbb{Z}})^{\ast} = C_{N-1}$ where $C_{N-1}$ is the cyclic group of order $N-1$.

Since the DFT at k = 0 is just the sum of the input data, we can rewrite the DFT as the following:

$X_k = x_0 + \sum_{n=1}^{N-1}x_n ⋅ e^{-iτkn/N}$ where $x_0 ≜ \sum x$
and the indices of n are present in the cyclic group $C_{N-1}$

By definition of a cyclic group, the cyclic group $C_{N-1}$ has a generator, and let this be $g$. ($g$ is a primitive root of $N$).

(A primitive root, $g$, of a prime number, $p$, is an integer, $g$, such that every integer coprime to p, $a$ (i.e. gcd(p, a) = 1) ≡ $g^z$ (mod p) for some integer $z$).

We can rewrite $n$ as: $g^{q_n}\\$
and $k$ as: $(g^{-1})^{p_k}$ = $g^{-p_k}$

$k ↦ p_k$ and $n ↦ q_n$ are bijections, since the order of $C_{N-1}$ is $N-1$ and $n,k\in\{0,1,2,...,N-2\}$

Thus we can rewrite the DFT as:

$X_{g^{-p}} = x_0 + \sum^{N-2}_{q=0}x_{g^q} ⋅ ω^{g^{-(p-q)}}$

This is now the cyclic convolution of $u(z) = ω^{g^{-z}}$ and $v(z) = x_{g^z}$

and using the convolution theorem, which states that:

$\mathcal{F}\{x \ast y\} = \mathcal{F}\{x\} ⋅ \mathcal{F}\{y\} $

To calculate the convolution of two lists, A and B, both of length M, pad them with zeros until they have a length L, where:

$L ≜ 2^{⌈log_2(M)⌉}$

$\mathcal{A} = A$ (padded with L - M 0s)$\\$
$\mathcal{B} = B$ (padded with L - M 0s)

then we can convolve $\mathcal{A}$ and $\mathcal{B}$ using a radix 2 FFT and radix 2 IFFT.

To then recover $A \ast B$, we can use:

$(A \ast B)_k = (\mathcal{A} \ast \mathcal{B})_k + (\mathcal{A} \ast \mathcal{B})_{k+m}$



In [12]:
# Finds the smallest primitive root of m (m must be prime)
function primroot(m::Int64)::Int64
    if !isprime(m)
        error("Argument 'm' must be a prime number")
    end

    if m == 2
        return 1 # 1 is the smallest primitive root of 2
    end

    P = keys(factor(m-1)) # Get prime factors of m-1
    for r = 2:(m-1)
        not_found = true
        for p in P
            if powermod(r, div(m-1, p), m) == 1
                not_found = false
            end
        end

        if not_found
            return r
        end
    end

    return 0
end

# Successive powers of g up to m - 1, all mod m
function sequence(g::Int64, m::Int64)::Vector{Int64}
    return [(g^i)%m for i ∈ 0:m-2]
end

# Use to generate the entire group (𝐙/p𝐙)* and successive powers (p is prime)
function generator(p::Int64)::Vector{Int64}
    if !isprime(p)
        throw(ArgumentError("p must be prime"))
    end

    return primroot(p), sequence(primroot(p), p)
end

# FFT convolution of u and v, where u and v are of the same length (uses padding)
function conv(u::Vector{T}, v::Vector{U})::Vector{Number} where {T <: Number, U <: Number}
    if length(u) != length(v)
        throw(ArgumentError("Input vectors must be of the same length"))
    end

    N = length(u)
    L = 2^ceil(log2(2N+1))
    ωₗ = exp(τ*im/L)

    fft_uₗ = radix2FFT(vcat(u, [0 for _ ∈ 1:L-N]), ωₗ)
    fft_vₗ = radix2FFT(vcat(v, [0 for _ ∈ 1:L-N]), ωₗ)

    # Convolution theorem: 𝓕{u(x) ∗ v(x)} = 𝓕{u(x)} . 𝓕{v(x)}
    uₗvₗ = radix2IFFT(fft_uₗ .* fft_vₗ)

    return [uₗvₗ[i] + uₗvₗ[i + N] for i in 1:N]
end

function rader_FFT(x::Vector{T}, ω::Union{Complex{A}, Nothing} = nothing)::Vector{ComplexF64} where {T <: Number, A <: Number}
    N = length(x)

    if !isprime(N)
        throw(ArgumentError("List must have a prime length"))
    end

    # ω ≜ exp(-im*τ/N) (if not already supplied, see IFFT for when different ω used) 
    if ω === nothing
        ω = exp(-τ*im/N)
    end

    g, g_seq = generator(N) # Generate g and its successive powers
    g⁻¹ = (g^(N-2))%N # Mulitplicative inverse of g (mod N)
    g⁻¹_seq = sequence(g⁻¹, N) # Successive powers of g⁻¹

    # u(z) = ω^g⁻ᶻ
    u = [ω^gₙ for gₙ ∈ g⁻¹_seq]

    # v(z) = x[gᶻ]
    v = [x[gᶻ+1] for gᶻ ∈ g_seq]

    uv = conv(u, v)

    fft = zeros(ComplexF64, N)
    fft[1] = sum(x) # FFT[0] = ∑ᵢ xᵢ (Julia is 1-indexed)

    # FFT[g⁻ʲ] = a₀ + (u * v)ⱼ (Julia is 1-indexed)
    for i in 1:N-1
        fft[g⁻¹_seq[i] + 1] = x[1] + uv[i]
    end

    return fft
end

5-element Vector{ComplexF64}:
                15.0 + 0.0im
 -2.5000000000000018 + 3.4409548011779334im
 -2.4999999999999996 + 0.8122992405822653im
 -2.5000000000000004 - 0.812299240582266im
 -2.4999999999999996 - 3.4409548011779347im

# Rader IFFT
The following code cell defines a rader IFFT.

From the reasoning presented for the radix 2 IFFT, we know that:

$IFFT(X) = FFT(X, \; ω = e^{iτ/N})$

and here we shall leverage this fact to make a rader IFFT

In [ ]:
function rader_IFFT(x::Vector{T}, ω::Union{Complex{U}, Nothing} = nothing)::Vector{ComplexF64} where {T <: Number, U <: Number}
    N = length(x)

    if !isprime(N)
        throw(ArgumentError("Input list must have a prime length"))
    end

    # Initialise ω to principal Nth root of unity if not supplied
    if ω === nothing
        ω = exp(τ*im/N)
    end

    return rader_FFT(x, 1/ω) ./ N
end